### БД из customer.csv и transaction.csv
Я так понимаю в csv те же тамблицы, что были в прошлом задании. Я вижу, что в задании говорится о двух таблицах. Но я же имею право спроектировать базу так как считаю правильным? Мне лень их перезаливать т_Т. Поэтому использую созданную в прошлом ДЗ базу:

![Компьютер](./images/scheme.svg)

#### 1. Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества.

In [ ]:
select job_industry_category, count(id) as customer_count from customers c group by job_industry_category;

![HW3.1.png](./images/HW3.1.png)

#### 2. Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности.

In [ ]:
SELECT 
    TO_CHAR(t.transaction_date, 'YYYY-MM') AS month,
    c.job_industry_category AS industry_category,
    SUM(p.list_price) AS total_transactions
FROM 
    transactions t
JOIN 
    customers c ON t.customer_id = c.id
JOIN 
    products p ON t.product_id = p.id
GROUP BY 
    month, c.job_industry_category
ORDER BY 
    month, c.job_industry_category;

![HW3.2.png](./images/HW3.2.png)

#### 3. Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT. 

In [ ]:
SELECT 
    p.brand AS brand,
    COUNT(t.id) AS online_order_count
FROM 
    transactions t
JOIN 
    customers c ON t.customer_id = c.id
JOIN 
    products p ON t.product_id = p.id
WHERE 
    t.online_order = TRUE
    AND t.order_status = 'Approved'
    AND c.job_industry_category = 'IT'
GROUP BY 
    p.brand
ORDER BY 
    online_order_count DESC;

![HW3.3.png](./images/HW3.3.png)

#### 4. Найти сумму, минимум, максимум и количество транзакций
Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат. 

In [ ]:
-- Использование group by

SELECT 
    c.id,
    c.first_name,
    c.last_name,
    SUM(p.list_price) AS total_transactions,
    MAX(p.list_price) AS max_transaction,
    MIN(p.list_price) AS min_transaction,
    COUNT(t.id) AS transaction_count
FROM transactions t
JOIN customers c ON t.customer_id = c.id
JOIN products p ON t.product_id = p.id
GROUP BY 
    c.id, c.first_name, c.last_name
ORDER BY 
    total_transactions DESC, transaction_count DESC;

![HW3.4.1.png](./images/HW3.4.1.png)

In [ ]:
-- Использование оконных функций

SELECT DISTINCT
    c.id,
    c.first_name,
    c.last_name,
    SUM(p.list_price) OVER (PARTITION BY c.id) AS total_transactions,
    MAX(p.list_price) OVER (PARTITION BY c.id) AS max_transaction,
    MIN(p.list_price) OVER (PARTITION BY c.id) AS min_transaction,
    COUNT(t.id) OVER (PARTITION BY c.id) AS transaction_count
FROM 
    transactions t
JOIN 
    customers c ON t.customer_id = c.id
JOIN 
    products p ON t.product_id = p.id
ORDER BY 
    total_transactions DESC, transaction_count DESC;

![HW3.4.2.png](./images/HW3.4.2.png)

### Сравнение
- Оба запроса возвращают одинаковые данные
- GROUP BY группирует данные и возвращает одну строку для каждого клиента.
- Оконные функции вычисляют значения для каждой строки, после чего используется DISTINCT для удаления дубликатов.

#### 5. Клиенты с минимальной и максимальной суммой транзакций
Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы.

In [ ]:
-- 1. Клиенты с максимальной суммой транзакций

WITH customer_transactions AS (
    SELECT 
        c.id,
        c.first_name,
        c.last_name,
        SUM(p.list_price) AS total_transactions
    FROM 
        transactions t
    JOIN 
        customers c ON t.customer_id = c.id
    JOIN 
        products p ON t.product_id = p.id
    GROUP BY 
        c.id, c.first_name, c.last_name
)
SELECT 
    first_name,
    last_name,
    total_transactions
FROM 
    customer_transactions
WHERE 
    total_transactions = (SELECT MAX(total_transactions) FROM customer_transactions);

![HW3.5.1.png](./images/HW3.5.1.png)

In [ ]:
-- 2. Клиенты с минимальной суммой транзакций

WITH customer_transactions AS (
    SELECT 
        c.id,
        c.first_name,
        c.last_name,
        SUM(p.list_price) AS total_transactions
    FROM 
        transactions t
    JOIN 
        customers c ON t.customer_id = c.id
    JOIN 
        products p ON t.product_id = p.id
    GROUP BY 
        c.id, c.first_name, c.last_name
)
SELECT 
    first_name,
    last_name,
    total_transactions
FROM 
    customer_transactions
WHERE 
    total_transactions = (SELECT MIN(total_transactions) FROM customer_transactions);

![HW3.5.2.png](./images/HW3.5.2.png)

#### 6. Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций.

In [ ]:
WITH first_transactions AS (
    SELECT 
        t.id as transaction_id,
        t.customer_id,
        t.transaction_date,
        t.product_id,
        t.online_order,
        t.order_status,
        ROW_NUMBER() OVER (
            PARTITION BY t.customer_id 
            ORDER BY t.transaction_date
        ) AS transaction_rank
    FROM 
        transactions t
)
SELECT 
    ft.transaction_id,
    ft.customer_id,
    c.first_name,
    c.last_name,
    ft.transaction_date,
    ft.product_id,
    ft.online_order,
    ft.order_status
FROM 
    first_transactions ft
JOIN 
    customers c ON ft.customer_id = c.id
WHERE 
    ft.transaction_rank = 1
ORDER BY 
    ft.customer_id;

![HW3.6.png](./images/HW3.6.png)

#### 7. Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях)

In [ ]:
WITH transaction_gaps AS (
    SELECT 
        customer_id,
        transaction_date - LAG(transaction_date) OVER (
            PARTITION BY customer_id 
            ORDER BY transaction_date
        ) AS days_diff
    FROM transactions
),
transaction_max AS (
    SELECT 
        customer_id,
        MAX(days_diff) AS max_interval_days
    FROM transaction_gaps
    WHERE days_diff IS NOT NULL
    GROUP BY customer_id
),
ranked_gaps AS (
    SELECT 
        *,
        DENSE_RANK() OVER (ORDER BY max_interval_days DESC) AS gap_rank
    FROM transaction_max
)
SELECT 
    rg.customer_id,
    rg.max_interval_days,
    c.*
FROM ranked_gaps rg
LEFT JOIN customers c ON c.id = rg.customer_id
WHERE rg.gap_rank = 1;

![HW3.7.png](./images/HW3.7.png)